In [8]:
# -*- coding: UTF-8 -*-
import pandas as pd
import numpy as np
import random
# !pip install streamlit -q # 如果后续需要
# !pip install gradio -q  # 如果后续需要

In [9]:
# -*- coding: UTF-8 -*-

def 计算基础代谢率(年龄, 性别, 身高_厘米, 体重_公斤):
    """根据Mifflin-St Jeor公式计算基础代谢率 (BMR)。"""
    if 性别.lower() == '男':
        基础代谢率 = (10 * 体重_公斤) + (6.25 * 身高_厘米) - (5 * 年龄) + 5
    elif 性别.lower() == '女':
        基础代谢率 = (10 * 体重_公斤) + (6.25 * 身高_厘米) - (5 * 年龄) - 161
    else:
        raise ValueError("性别必须是 '男' 或 '女'")
    return 基础代谢率

def 计算每日总能量消耗(基础代谢率, 活动水平):
    """计算每日总能量消耗 (TDEE)。"""
    活动系数表 = {
        "久坐": 1.2,
        "轻度活动": 1.375,
        "中度活动": 1.55,
        "积极活动": 1.725,
        "非常积极": 1.9
    }
    if 活动水平.lower() not in 活动系数表:
        raise ValueError(f"无效的活动水平。请从以下选项中选择: {', '.join(活动系数表.keys())}")
    return 基础代谢率 * 活动系数表[活动水平.lower()]

def 获取宏量营养素推荐(每日总能量消耗, 体重_公斤, 目标=["维持"], 每公斤体重蛋白质克数=1.6, 脂肪供能百分比=0.25, 最大蛋白质供能占比=0.35):
    """
    计算宏量营养素推荐。
    新增: 最大蛋白质供能占比，用于限制蛋白质摄入上限。
    """
    调整后每日总能量消耗 = 每日总能量消耗
    if "减脂" in 目标 and "增肌" not in 目标:
        调整后每日总能量消耗 *= 0.8
    elif "增肌" in 目标:
        调整后每日总能量消耗 *= 1.1
        每公斤体重蛋白质克数 = min(max(每公斤体重蛋白质克数, 1.8), 2.2)

    初步蛋白质克数 = 每公斤体重蛋白质克数 * 体重_公斤
    最大允许蛋白质热量 = 调整后每日总能量消耗 * 最大蛋白质供能占比
    最大允许蛋白质克数_按占比 = 最大允许蛋白质热量 / 4
    最终蛋白质克数 = min(初步蛋白质克数, 最大允许蛋白质克数_按占比)
    最终蛋白质热量 = 最终蛋白质克数 * 4

    脂肪热量 = 调整后每日总能量消耗 * 脂肪供能百分比
    脂肪热量 = min(脂肪热量, 调整后每日总能量消耗 - 最终蛋白质热量)
    if 脂肪热量 < 0: 脂肪热量 = 0

    碳水化合物热量 = 调整后每日总能量消耗 - 最终蛋白质热量 - 脂肪热量
    蛋白质_克 = 最终蛋白质克数
    脂肪_克 = 脂肪热量 / 9
    碳水化合物_克 = 碳水化合物热量 / 4

    if 碳水化合物_克 < 50: # 保证最低碳水50g
        碳水化合物_克 = 50
        碳水化合物热量 = 碳水化合物_克 * 4
        剩余热量给脂肪 = 调整后每日总能量消耗 - 最终蛋白质热量 - 碳水化合物热量
        脂肪热量 = max(0, 剩余热量给脂肪)
        脂肪_克 = 脂肪热量 / 9

    return {
        "调整后每日总能量消耗_千卡": round(调整后每日总能量消耗),
        "蛋白质_克": round(蛋白质_克),
        "碳水化合物_克": round(碳水化合物_克),
        "脂肪_克": round(脂肪_克)
    }

# def 推荐食物类别 ... (这个函数目前未使用，可以省略或保留)

In [10]:
# -*- coding: UTF-8 -*-
print("--- 请提供您的详细信息 ---")
try:
    # 为了复现上次 144g 蛋白质的目标，请确保输入与上次一致的能产生该目标的体重等数据
    年龄 = int(input("年龄 (岁): "))
    性别 = input("性别 (男/女): ").strip()
    身高_厘米 = float(input("身高 (厘米): "))
    体重_公斤 = float(input("体重 (公斤): ")) # <--- 注意这里的体重输入
    活动水平 = input("活动水平 (久坐/轻度活动/中度活动/积极活动/非常积极): ").strip()
    目标_字符串 = input("主要目标 (用逗号分隔, 例如: 减脂,增肌,维持): ").strip()
    用户目标 = [目标.strip().lower() for 目标 in 目标_字符串.split(',')]

    print("\n--- 感谢您的输入！正在计算... ---")
    输入有效 = True
except ValueError:
    print("\n输入错误：年龄、身高、体重必须是数字。请重新运行单元格并正确输入。")
    输入有效 = False
except Exception as e:
    print(f"\n发生未知输入错误: {e}")
    输入有效 = False

--- 请提供您的详细信息 ---
年龄 (岁): 50
性别 (男/女): 男
身高 (厘米): 170
体重 (公斤): 90
活动水平 (久坐/轻度活动/中度活动/积极活动/非常积极): 久坐
主要目标 (用逗号分隔, 例如: 减脂,增肌,维持): 减脂

--- 感谢您的输入！正在计算... ---


In [4]:
# -*- coding: UTF-8 -*-
if '输入有效' in locals() and 输入有效:
    try:
        用户_基础代谢率 = 计算基础代谢率(年龄, 性别, 身高_厘米, 体重_公斤)
        用户_每日总能量消耗 = 计算每日总能量消耗(用户_基础代谢率, 活动水平)
        宏量营养素推荐结果 = 获取宏量营养素推荐(用户_每日总能量消耗, 体重_公斤, 用户目标)

        print(f"\n--- 您的预估营养需求 ---")
        print(f"基础代谢率 (BMR): {用户_基础代谢率:.0f} 千卡/天")
        print(f"每日总能量消耗 (TDEE): {用户_每日总能量消耗:.0f} 千卡/天")
        print(f"调整后每日热量目标 (根据您的目标): {宏量营养素推荐结果['调整后每日总能量消耗_千卡']} 千卡/天")

        print(f"\n--- 推荐宏量营养素摄入 ---")
        print(f"蛋白质: {宏量营养素推荐结果['蛋白质_克']} 克/天")
        print(f"碳水化合物: {宏量营养素推荐结果['碳水化合物_克']} 克/天")
        print(f"脂肪: {宏量营养素推荐结果['脂肪_克']} 克/天")

    except ValueError as e:
        print(f"\n计算错误: {e}")
    except Exception as e:
        print(f"\n发生未知计算错误: {e}")
        宏量营养素推荐结果 = None
elif '输入有效' not in locals():
     print("\n请先运行上一个包含用户输入的单元格。")
     宏量营养素推荐结果 = None
elif not 输入有效:
     print("\n由于之前的输入错误，无法进行计算。请修正输入并重新运行。")
     宏量营养素推荐结果 = None


--- 您的预估营养需求 ---
基础代谢率 (BMR): 1718 千卡/天
每日总能量消耗 (TDEE): 2061 千卡/天
调整后每日热量目标 (根据您的目标): 1649 千卡/天

--- 推荐宏量营养素摄入 ---
蛋白质: 144 克/天
碳水化合物: 165 克/天
脂肪: 46 克/天


In [11]:
# -*- coding: UTF-8 -*-
食物数据库_原始列表 = [
    # 优质蛋白质来源
    {"名称": "鸡胸肉(生)", "类别": "蛋白质", "蛋白质": 22, "碳水": 0, "脂肪": 2, "单位": "克", "单次最大推荐_克": 200, "最小推荐_克": 50},
    {"名称": "三文鱼(生)", "类别": "蛋白质", "蛋白质": 20, "碳水": 0, "脂肪": 13, "单位": "克", "单次最大推荐_克": 120, "最小推荐_克": 50}, # 调整
    {"名称": "鸡蛋(整个)", "类别": "蛋白质", "蛋白质": 13, "碳水": 1, "脂肪": 11, "单位": "个(约50克)", "单次最大推荐_克": 150, "最小推荐_克": 50},
    {"名称": "豆腐(老)", "类别": "蛋白质", "蛋白质": 8, "碳水": 2, "脂肪": 4, "单位": "克", "单次最大推荐_克": 250, "最小推荐_克": 100},
    {"名称": "希腊酸奶(原味0脂)", "类别": "蛋白质", "蛋白质": 10, "碳水": 4, "脂肪": 0, "单位": "克", "单次最大推荐_克": 200, "最小推荐_克": 100},
    {"名称": "乳清蛋白粉", "类别": "蛋白质", "蛋白质": 80, "碳水": 5, "脂肪": 7, "单位": "克", "单次最大推荐_克": 60, "最小推荐_克": 20},

    # 复合碳水化合物来源
    {"名称": "糙米(生重)", "类别": "碳水", "蛋白质": 7, "碳水": 77, "脂肪": 3, "单位": "克", "单次最大推荐_克": 100, "最小推荐_克": 30},
    {"名称": "燕麦片(生)", "类别": "碳水", "蛋白质": 17, "碳水": 66, "脂肪": 7, "单位": "克", "单次最大推荐_克": 80, "最小推荐_克": 30},
    {"名称": "红薯(生)", "类别": "碳水", "蛋白质": 1.6, "碳水": 20, "脂肪": 0.1, "单位": "克", "单次最大推荐_克": 250, "最小推荐_克": 100}, # 调整
    {"名称": "全麦面包", "类别": "碳水", "蛋白质": 13, "碳水": 41, "脂肪": 3, "单位": "片(约35克)", "单次最大推荐_克": 140, "最小推荐_克": 35},
    {"名称": "藜麦(生重)", "类别": "碳水", "蛋白质": 14, "碳水": 64, "脂肪": 6, "单位": "克", "单次最大推荐_克": 80, "最小推荐_克": 30},
    {"名称": "香蕉", "类别": "碳水", "蛋白质": 1.1, "碳水":22.8, "脂肪":0.3, "单位":"个(约120克净重)", "单次最大推荐_克":120, "最小推荐_克":120},


    # 健康脂肪来源
    {"名称": "牛油果", "类别": "脂肪", "蛋白质": 2, "碳水": 9, "脂肪": 15, "单位": "个(约150克净重)", "单次最大推荐_克": 150, "最小推荐_克": 75},
    {"名称": "杏仁", "类别": "脂肪", "蛋白质": 21, "碳水": 22, "脂肪": 49, "单位": "克", "单次最大推荐_克": 30, "最小推荐_克": 10},
    {"名称": "核桃", "类别": "脂肪", "蛋白质": 15, "碳水": 14, "脂肪": 65, "单位": "克", "单次最大推荐_克": 30, "最小推荐_克": 10},
    {"名称": "橄榄油", "类别": "脂肪", "蛋白质": 0, "碳水": 0, "脂肪": 100, "单位": "克", "单次最大推荐_克": 20, "最小推荐_克": 5},

    # 蔬菜
    {"名称": "西兰花(生)", "类别": "蔬菜", "蛋白质": 2.8, "碳水": 7, "脂肪": 0.4, "单位": "克", "单次最大推荐_克": 250, "最小推荐_克": 100},
    {"名称": "菠菜(生)", "类别": "蔬菜", "蛋白质": 2.9, "碳水": 3.6, "脂肪": 0.4, "单位": "克", "单次最大推荐_克": 250, "最小推荐_克": 100},
    {"名称": "番茄(生)", "类别": "蔬菜", "蛋白质": 0.9, "碳水": 3.9, "脂肪": 0.2, "单位": "克", "单次最大推荐_克": 200, "最小推荐_克": 100},
    {"名称": "黄瓜(生)", "类别": "蔬菜", "蛋白质": 0.7, "碳水": 3.6, "脂肪": 0.1, "单位": "克", "单次最大推荐_克": 200, "最小推荐_克": 100},
    {"名称": "胡萝卜(生)", "类别": "蔬菜", "蛋白质": 0.9, "碳水": 9.6, "脂肪": 0.2, "单位": "克", "单次最大推荐_克": 150, "最小推荐_克": 50},
]

食物数据表 = pd.DataFrame(食物数据库_原始列表)

def 标准化食物单位(df_input):
    df = df_input.copy()
    for index, row in df.iterrows():
        if row['单位'] == '个(约50克)' and row['名称'] == '鸡蛋(整个)':
            营养素列 = ['蛋白质', '碳水', '脂肪']
            df.loc[index, 营养素列] = df.loc[index, 营养素列] * (100/50)
        elif row['单位'] == '片(约35克)' and row['名称'] == '全麦面包':
            营养素列 = ['蛋白质', '碳水', '脂肪']
            df.loc[index, 营养素列] = df.loc[index, 营养素列] * (100/35)
        elif row['单位'] == '个(约150克净重)' and row['名称'] == '牛油果':
            营养素列 = ['蛋白质', '碳水', '脂肪']
            df.loc[index, 营养素列] = df.loc[index, 营养素列] * (100/150)
        elif row['单位'] == '个(约120克净重)' and row['名称'] == '香蕉':
            营养素列 = ['蛋白质', '碳水', '脂肪']
            df.loc[index, 营养素列] = df.loc[index, 营养素列] * (100/120)

    for col in ['蛋白质', '碳水', '脂肪', '单次最大推荐_克', '最小推荐_克']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

食物数据表_标准化 = 标准化食物单位(食物数据表)
# print("食物数据库加载完毕，标准化完成。")

In [12]:
# -*- coding: UTF-8 -*-

def 计算当前已选总营养(当前已选_dict, 食物表_df):
    总P, 总C, 总F = 0, 0, 0
    for 名称, 份量 in 当前已选_dict.items():
        if 份量 <=0 : continue
        try:
            信息 = 食物表_df[食物表_df['名称'] == 名称].iloc[0]
            总P += 信息['蛋白质'] * (份量 / 100)
            总C += 信息['碳水'] * (份量 / 100)
            总F += 信息['脂肪'] * (份量 / 100)
        except IndexError:
            print(f"警告: 食物 '{名称}' 在数据库中未找到，无法计算其营养。")
    return 总P, 总C, 总F


def 选择食材方案(目标P_克, 目标C_克, 目标F_克, 食物数据_df, 用户偏好=None, 避免食物=None):
    """
    改进的启发式算法，尝试更平衡地选择食材。
    调整了脂肪超标控制和蔬菜碳水预算。
    """
    已选食材 = {}
    剩余P, 剩余C, 剩余F = 目标P_克, 目标C_克, 目标F_克

    if 避免食物:
        可用食物数据 = 食物数据_df[~食物数据_df['名称'].isin(避免食物)].copy()
    else:
        可用食物数据 = 食物数据_df.copy()

    if 可用食物数据.empty:
        print("警告：根据您的避免清单，没有可用食物可选！")
        return {}, 剩余P, 剩余C, 剩余F

    最大迭代次数 = 5
    for iteration in range(最大迭代次数):
        有食材被添加本轮 = False

        # --- 1. 蛋白质 ---
        if 剩余P > 目标P_克 * 0.05:
            高蛋白食物 = 可用食物数据[可用食物数据['类别'] == '蛋白质'].sort_values(by='蛋白质', ascending=False)
            for _, 食物 in 高蛋白食物.iterrows():
                if 剩余P <= 目标P_克 * 0.05 : break
                if 食物['蛋白质'] <=0 : continue

                理论所需 = (剩余P / (食物['蛋白质'] / 100))
                单次最大 = 食物.get('单次最大推荐_克', 150)
                if pd.isna(单次最大):单次最大 = 150
                if 食物['名称'] == '乳清蛋白粉': 单次最大 = min(单次最大, 60)

                添加量 = min(理论所需, 单次最大)
                添加量 = max(0, round(添加量))
                最小添加 = 食物.get('最小推荐_克', 10)
                if pd.isna(最小添加):最小添加 = 10
                if 添加量 < 最小添加 : 添加量 = 0

                if 添加量 > 0:
                    temp_已选 = 已选食材.copy()
                    if 食物['名称'] in temp_已选: temp_已选[食物['名称']] += 添加量
                    else: temp_已选[食物['名称']] = 添加量
                    预估总P, 预估总C, 预估总F = 计算当前已选总营养(temp_已选, 可用食物数据)

                    # 调整脂肪超标控制
                    脂肪允许超标因子 = 1.10
                    脂肪附带量阈值 = 1 # 每100g食物脂肪含量低于此值

                    if (预估总P < 目标P_克 * 1.15 or 剩余P > 0) and \
                       (预估总C < 目标C_克 * 1.15 or 剩余C > 0 or 食物['碳水'] < 5) and \
                       (预估总F < 目标F_克 * 脂肪允许超标因子 or 剩余F > -(目标F_克 * 0.05) or 食物['脂肪'] < 脂肪附带量阈值) :
                        已选食材 = temp_已选
                        剩余P -= 食物['蛋白质'] * (添加量/100)
                        剩余C -= 食物['碳水'] * (添加量/100)
                        剩余F -= 食物['脂肪'] * (添加量/100)
                        有食材被添加本轮 = True

        # --- 2. 脂肪 ---
        if 剩余F > 目标F_克 * 0.1:
            健康脂肪食物 = 可用食物数据[可用食物数据['类别'] == '脂肪'].sort_values(by='脂肪', ascending=False)
            for _, 食物 in 健康脂肪食物.iterrows():
                if 剩余F <= 目标F_克 * 0.1 : break
                if 食物['脂肪'] <= 0: continue

                理论所需 = (剩余F / (食物['脂肪'] / 100))
                单次最大 = 食物.get('单次最大推荐_克', 50)
                if pd.isna(单次最大):单次最大 = 50
                if 食物['名称'] == '橄榄油': 单次最大 = min(单次最大, 20)

                添加量 = min(理论所需, 单次最大)
                添加量 = max(0, round(添加量))
                最小添加 = 食物.get('最小推荐_克', 5)
                if pd.isna(最小添加):最小添加 = 5
                if 添加量 < 最小添加 : 添加量 = 0

                if 添加量 > 0:
                    temp_已选 = 已选食材.copy()
                    if 食物['名称'] in temp_已选: temp_已选[食物['名称']] += 添加量
                    else: temp_已选[食物['名称']] = 添加量
                    预估总P, 预估总C, 预估总F = 计算当前已选总营养(temp_已选, 可用食物数据)

                    # 脂肪是主攻，P和C是副作用
                    if (预估总F < 目标F_克 * 1.15 or 剩余F > 0) and \
                       (预估总P < 目标P_克 * 1.10 or 剩余P > -(目标P_克*0.05) or 食物['蛋白质'] < 1) and \
                       (预估总C < 目标C_克 * 1.10 or 剩余C > -(目标C_克*0.05) or 食物['碳水'] < 2) :
                        已选食材 = temp_已选
                        剩余P -= 食物['蛋白质'] * (添加量/100)
                        剩余C -= 食物['碳水'] * (添加量/100)
                        剩余F -= 食物['脂肪'] * (添加量/100)
                        有食材被添加本轮 = True

        # --- 3. 碳水化合物 ---
        if 剩余C > 目标C_克 * 0.05:
            复合碳水食物 = 可用食物数据[可用食物数据['类别'] == '碳水'].sort_values(by='碳水', ascending=False)
            for _, 食物 in 复合碳水食物.iterrows():
                if 剩余C <= 目标C_克 * 0.05 : break
                if 食物['碳水'] <= 0: continue

                理论所需 = (剩余C / (食物['碳水'] / 100))
                单次最大 = 食物.get('单次最大推荐_克', 150)
                if pd.isna(单次最大):单次最大 = 150

                添加量 = min(理论所需, 单次最大)
                添加量 = max(0, round(添加量))
                最小添加 = 食物.get('最小推荐_克', 20)
                if pd.isna(最小添加):最小添加 = 20
                if 添加量 < 最小添加 : 添加量 = 0

                if 添加量 > 0:
                    temp_已选 = 已选食材.copy()
                    if 食物['名称'] in temp_已选: temp_已选[食物['名称']] += 添加量
                    else: temp_已选[食物['名称']] = 添加量
                    预估总P, 预估总C, 预估总F = 计算当前已选总营养(temp_已选, 可用食物数据)

                    脂肪允许超标因子 = 1.10 # 碳水阶段对脂肪的副作用控制
                    脂肪附带量阈值 = 1

                    if (预估总C < 目标C_克 * 1.15 or 剩余C > 0) and \
                       (预估总P < 目标P_克 * 1.15 or 剩余P > 0 or 食物['蛋白质'] < 5) and \
                       (预估总F < 目标F_克 * 脂肪允许超标因子 or 剩余F > -(目标F_克 * 0.05) or 食物['脂肪'] < 脂肪附带量阈值) :
                        已选食材 = temp_已选
                        剩余P -= 食物['蛋白质'] * (添加量/100)
                        剩余C -= 食物['碳水'] * (添加量/100)
                        剩余F -= 食物['脂肪'] * (添加量/100)
                        有食材被添加本轮 = True

        if not 有食材被添加本轮 and iteration > 0 :
            break
        current_total_P, current_total_C, current_total_F = 计算当前已选总营养(已选食材, 可用食物数据)
        if abs(current_total_P - 目标P_克) < 目标P_克 * 0.1 and \
           abs(current_total_C - 目标C_克) < 目标C_克 * 0.1 and \
           abs(current_total_F - 目标F_克) < 目标F_克 * 0.15: # 脂肪的允许偏差略大一点
            break

    # --- 4. 添加蔬菜 ---
    蔬菜类 = 可用食物数据[可用食物数据['类别'] == '蔬菜']
    已选蔬菜种类数 = 0
    if not 蔬菜类.empty:
        非蔬菜碳水 = 0
        for 名称, 份量 in 已选食材.items():
            食物信息 = 可用食物数据[可用食物数据['名称'] == 名称].iloc[0]
            if 食物信息['类别'] != '蔬菜':
                非蔬菜碳水 += 食物信息['碳水'] * (份量 / 100)

        剩余碳水给蔬菜 = 目标C_克 * 1.05 - 非蔬菜碳水 # 调整：蔬菜让总碳水超目标5%

        for _ in range(min(len(蔬菜类), 3)):
            if 已选蔬菜种类数 >= 2 and 剩余碳水给蔬菜 < 10:
                break
            未选蔬菜 = 蔬菜类[~蔬菜类['名称'].isin(已选食材.keys())]
            if 未选蔬菜.empty: break

            随机蔬菜 = 未选蔬菜.sample(1).iloc[0]
            单次最大蔬菜 = 随机蔬菜.get('单次最大推荐_克', 200)
            if pd.isna(单次最大蔬菜):单次最大蔬菜 = 200
            最小蔬菜添加 = 随机蔬菜.get('最小推荐_克', 50)
            if pd.isna(最小蔬菜添加):最小蔬菜添加 = 50

            添加量 = 单次最大蔬菜
            if 随机蔬菜['碳水'] > 0:
                理论可加蔬菜量据碳水 = (剩余碳水给蔬菜 / (随机蔬菜['碳水'] / 100))
                添加量 = min(添加量, 理论可加蔬菜量据碳水)
            添加量 = min(添加量, 单次最大蔬菜)
            添加量 = max(0, round(添加量))

            if 添加量 >= 最小蔬菜添加 :
                temp_已选 = 已选食材.copy()
                if 随机蔬菜['名称'] in temp_已选: temp_已选[随机蔬菜['名称']] += 添加量
                else: temp_已选[随机蔬菜['名称']] = 添加量
                预估总P_加菜, 预估总C_加菜, 预估总F_加菜 = 计算当前已选总营养(temp_已选, 可用食物数据)

                允许超标因子_蔬菜 = 1.18 # 蔬菜带来的宏量超标容忍度
                if (预估总C_加菜 < 目标C_克 * 允许超标因子_蔬菜 or 剩余C > 0 ) and \
                   (预估总P_加菜 < 目标P_克 * 允许超标因子_蔬菜 or 剩余P > 0 ) and \
                   (预估总F_加菜 < 目标F_克 * 允许超标因子_蔬菜 or 剩余F > 0 ) :
                    已选食材 = temp_已选
                    剩余碳水给蔬菜 -= 随机蔬菜['碳水'] * (添加量/100)
                    已选蔬菜种类数 +=1

    已选食材_最终 = {}
    for name, amount in 已选食材.items():
        food_info = 可用食物数据[可用食物数据['名称'] == name].iloc[0]
        min_rec = food_info.get('最小推荐_克', 10)
        if pd.isna(min_rec): min_rec = 10
        if amount >= min_rec or (food_info['类别'] == '脂肪' and amount > 1) : # 脂肪大于1克就保留
             已选食材_最终[name] = round(amount)

    final_P_total, final_C_total, final_F_total = 计算当前已选总营养(已选食材_最终, 可用食物数据)
    final_剩余P = 目标P_克 - final_P_total
    final_剩余C = 目标C_克 - final_C_total
    final_剩余F = 目标F_克 - final_F_total

    return 已选食材_最终, round(final_剩余P), round(final_剩余C), round(final_剩余F)

In [13]:
# -*- coding: UTF-8 -*-
if '宏量营养素推荐结果' in locals() and 宏量营养素推荐结果 is not None and 输入有效:
    目标P = 宏量营养素推荐结果["蛋白质_克"]
    目标C = 宏量营养素推荐结果["碳水化合物_克"]
    目标F = 宏量营养素推荐结果["脂肪_克"]

    用户偏好列表 = None
    避免食物列表 = None

    print(f"\n--- 正在为您搭配食材 (目标 P:{目标P}g, C:{目标C}g, F:{目标F}g) ---")

    if '食物数据表_标准化' not in globals():
        print("错误：食物数据库未加载。请先运行包含食物数据库定义的单元格 (单元格 5)。")
    else:
        推荐食材方案, 最终剩余P, 最终剩余C, 最终剩余F = 选择食材方案(
            目标P, 目标C, 目标F, 食物数据表_标准化, 用户偏好列表, 避免食物列表
        )

        print("\n--- 每日推荐食材方案 (仅供参考，份量为生重或原始单位) ---")
        if not 推荐食材方案:
            print("未能生成有效的食材方案。可能是食物数据库过小或目标难以满足。")
        else:
            总计P_方案, 总计C_方案, 总计F_方案 = 0, 0, 0
            if isinstance(推荐食材方案, dict):
                for 食物名称, 份量 in 推荐食材方案.items():
                    print(f"- {食物名称}: {份量} 克")
                    try:
                        食物信息 = 食物数据表_标准化[食物数据表_标准化['名称'] == 食物名称].iloc[0]
                        总计P_方案 += 食物信息['蛋白质'] * (份量 / 100)
                        总计C_方案 += 食物信息['碳水'] * (份量 / 100)
                        总计F_方案 += 食物信息['脂肪'] * (份量 / 100)
                    except IndexError:
                        print(f"警告: 食物 '{食物名称}' 在数据库中未找到，无法计入方案总营养。")

                print(f"\n--- 方案总计营养 (估算) ---")
                print(f"蛋白质: {总计P_方案:.1f} 克 (目标: {目标P} 克, 偏差: {总计P_方案 - 目标P:.1f} 克)")
                print(f"碳水化合物: {总计C_方案:.1f} 克 (目标: {目标C} 克, 偏差: {总计C_方案 - 目标C:.1f} 克)")
                print(f"脂肪: {总计F_方案:.1f} 克 (目标: {目标F} 克, 偏差: {总计F_方案 - 目标F:.1f} 克)")
                总热量_方案 = 总计P_方案*4 + 总计C_方案*4 + 总计F_方案*9
                目标总热量 = 宏量营养素推荐结果['调整后每日总能量消耗_千卡']
                print(f"总热量估算: {总热量_方案:.0f} 千卡 (目标热量: {目标总热量:.0f} 千卡, 偏差: {总热量_方案 - 目标总热量:.0f} 千卡)")

                protein_diff_percent = abs(总计P_方案 - 目标P) / 目标P if 目标P > 0 else 0
                carb_diff_percent = abs(总计C_方案 - 目标C) / 目标C if 目标C > 0 else 0
                fat_diff_percent = abs(总计F_方案 - 目标F) / 目标F if 目标F > 0 else 0
                calorie_diff_percent = abs(总热量_方案 - 目标总热量) / 目标总热量 if 目标总热量 > 0 else 0

                if protein_diff_percent > 0.15 or carb_diff_percent > 0.15 or fat_diff_percent > 0.20 or calorie_diff_percent > 0.12: # 热量偏差阈值略微放宽到12%
                    print("\n提示: 食材搭配结果与目标仍有一定差距。")
                else:
                    print("\n食材搭配结果与目标较为接近！")
            else:
                print("推荐食材方案格式不正确。")

elif '宏量营养素推荐结果' in locals() and 宏量营养素推荐结果 is None :
    print("\n由于之前的计算错误，无法进行食材搭配。")
else:
    print("\n请先完成前面的营养需求计算步骤。")


--- 正在为您搭配食材 (目标 P:144g, C:165g, F:46g) ---

--- 每日推荐食材方案 (仅供参考，份量为生重或原始单位) ---
- 乳清蛋白粉: 60 克
- 鸡蛋(整个): 150 克
- 鸡胸肉(生): 200 克
- 三文鱼(生): 65 克
- 红薯(生): 500 克
- 香蕉: 240 克
- 胡萝卜(生): 150 克
- 西兰花(生): 104 克

--- 方案总计营养 (估算) ---
蛋白质: 158.5 克 (目标: 144 克, 偏差: 14.5 克)
碳水化合物: 173.3 克 (目标: 165 克, 偏差: 8.3 克)
脂肪: 51.5 克 (目标: 46 克, 偏差: 5.5 克)
总热量估算: 1790 千卡 (目标热量: 1649 千卡, 偏差: 141 千卡)

食材搭配结果与目标较为接近！


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
import pandas as pd

# --- 确保以下 食物数据库_原始列表 与您最终版本一致 ---
食物数据库_原始列表 = [
    # 优质蛋白质来源 (此处省略，请确保这是您最终使用的完整列表)
    {"名称": "鸡胸肉(生)", "类别": "蛋白质", "蛋白质": 22, "碳水": 0, "脂肪": 2, "单位": "克", "单次最大推荐_克": 200, "最小推荐_克": 50},
    {"名称": "三文鱼(生)", "类别": "蛋白质", "蛋白质": 20, "碳水": 0, "脂肪": 13, "单位": "克", "单次最大推荐_克": 120, "最小推荐_克": 50},
    {"名称": "鸡蛋(整个)", "类别": "蛋白质", "蛋白质": 13, "碳水": 1, "脂肪": 11, "单位": "个(约50克)", "单次最大推荐_克": 150, "最小推荐_克": 50},
    {"名称": "豆腐(老)", "类别": "蛋白质", "蛋白质": 8, "碳水": 2, "脂肪": 4, "单位": "克", "单次最大推荐_克": 250, "最小推荐_克": 100},
    {"名称": "希腊酸奶(原味0脂)", "类别": "蛋白质", "蛋白质": 10, "碳水": 4, "脂肪": 0, "单位": "克", "单次最大推荐_克": 200, "最小推荐_克": 100},
    {"名称": "乳清蛋白粉", "类别": "蛋白质", "蛋白质": 80, "碳水": 5, "脂肪": 7, "单位": "克", "单次最大推荐_克": 60, "最小推荐_克": 20},
    {"名称": "糙米(生重)", "类别": "碳水", "蛋白质": 7, "碳水": 77, "脂肪": 3, "单位": "克", "单次最大推荐_克": 100, "最小推荐_克": 30},
    {"名称": "燕麦片(生)", "类别": "碳水", "蛋白质": 17, "碳水": 66, "脂肪": 7, "单位": "克", "单次最大推荐_克": 80, "最小推荐_克": 30},
    {"名称": "红薯(生)", "类别": "碳水", "蛋白质": 1.6, "碳水": 20, "脂肪": 0.1, "单位": "克", "单次最大推荐_克": 250, "最小推荐_克": 100},
    {"名称": "全麦面包", "类别": "碳水", "蛋白质": 13, "碳水": 41, "脂肪": 3, "单位": "片(约35克)", "单次最大推荐_克": 140, "最小推荐_克": 35},
    {"名称": "藜麦(生重)", "类别": "碳水", "蛋白质": 14, "碳水": 64, "脂肪": 6, "单位": "克", "单次最大推荐_克": 80, "最小推荐_克": 30},
    {"名称": "香蕉", "类别": "碳水", "蛋白质": 1.1, "碳水":22.8, "脂肪":0.3, "单位":"个(约120克净重)", "单次最大推荐_克":120, "最小推荐_克":120},
    {"名称": "牛油果", "类别": "脂肪", "蛋白质": 2, "碳水": 9, "脂肪": 15, "单位": "个(约150克净重)", "单次最大推荐_克": 150, "最小推荐_克": 75},
    {"名称": "杏仁", "类别": "脂肪", "蛋白质": 21, "碳水": 22, "脂肪": 49, "单位": "克", "单次最大推荐_克": 30, "最小推荐_克": 10},
    {"名称": "核桃", "类别": "脂肪", "蛋白质": 15, "碳水": 14, "脂肪": 65, "单位": "克", "单次最大推荐_克": 30, "最小推荐_克": 10},
    {"名称": "橄榄油", "类别": "脂肪", "蛋白质": 0, "碳水": 0, "脂肪": 100, "单位": "克", "单次最大推荐_克": 20, "最小推荐_克": 5},
    {"名称": "西兰花(生)", "类别": "蔬菜", "蛋白质": 2.8, "碳水": 7, "脂肪": 0.4, "单位": "克", "单次最大推荐_克": 250, "最小推荐_克": 100},
    {"名称": "菠菜(生)", "类别": "蔬菜", "蛋白质": 2.9, "碳水": 3.6, "脂肪": 0.4, "单位": "克", "单次最大推荐_克": 250, "最小推荐_克": 100},
    {"名称": "番茄(生)", "类别": "蔬菜", "蛋白质": 0.9, "碳水": 3.9, "脂肪": 0.2, "单位": "克", "单次最大推荐_克": 200, "最小推荐_克": 100},
    {"名称": "黄瓜(生)", "类别": "蔬菜", "蛋白质": 0.7, "碳水": 3.6, "脂肪": 0.1, "单位": "克", "单次最大推荐_克": 200, "最小推荐_克": 100},
    {"名称": "胡萝卜(生)", "类别": "蔬菜", "蛋白质": 0.9, "碳水": 9.6, "脂肪": 0.2, "单位": "克", "单次最大推荐_克": 150, "最小推荐_克": 50},
]
# --- 列表定义结束 ---

try:
    if '食物数据库_原始列表' in globals() and isinstance(食物数据库_原始列表, list):
        df_for_csv = pd.DataFrame(食物数据库_原始列表)
        csv_filename = "food_database.csv"
        csv_save_path_colab = f"/content/{csv_filename}" # 先保存在Colab临时环境
        csv_save_path_drive = f"/content/drive/My Drive/{csv_filename}" # 之后要复制到的Drive路径

        df_for_csv.to_csv(csv_save_path_colab, index=False, encoding='utf-8-sig')
        print(f"食物数据库已保存到Colab临时路径: {csv_save_path_colab}")

        # 将CSV文件从Colab环境复制到Google Drive
        import shutil
        shutil.copy(csv_save_path_colab, csv_save_path_drive)
        print(f"食物数据库已成功复制到Google Drive: {csv_save_path_drive}")
    else:
        print("错误：'食物数据库_原始列表' 未定义或不是一个列表。请先运行定义该列表的单元格。")

except Exception as e:
    print(f"保存食物数据库到CSV时发生错误: {e}")

食物数据库已保存到Colab临时路径: /content/food_database.csv
食物数据库已成功复制到Google Drive: /content/drive/My Drive/food_database.csv
